## Chief Technology Operations Project - CTO_Bank Project

The CTO_Bank Project uses data sourced from [dataset](https://www.kaggle.com/datasets/shivamb/bank-customer-segmentation)

The aim of this project is to create a machine learning model that will recomend products and services to a customer, predict a customers salarary and 
predict a bank customers likelihood of purchasing products or services offered by the bank (CTO_Bank). The drivers of this prediction will be evaluated 
from the following data exploration the frequency a user transacts, the average spend of the user per transaction, customer account balance average 
over the months or days.

The product and services offered by the bank include 
1. Loans
2. Car Insurance 
3. Medical Aid
3. Phone Insurance
4. Car Road Side Assistance
5. Dental Benefits
...
products and services will vary from this document since products and services will be created dynamically on the user interface.

At project completion the Machine Learning Models will be deployed to the Web App. These models will recommend products and services to a user, on the 
on  the admin panel, it should predict a users expected salary based on their account balance and also forecast the users spending.

Visit to view project: 
                     bongagprojects.site / bongagprojects.site:9091 (Work In Progress)

In [35]:
# Now using the new dataset csv file
import pandas as pd
df = pd.read_csv("new_dataset.csv")
df.head(5)

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (R),Age
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,1970-01-01 00:00:00.000143207,25.0,30.0
1,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,1970-01-01 00:00:00.000142712,459.0,28.0
2,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,1970-01-01 00:00:00.000181156,1762.5,36.0
3,T7,C7126560,1992-01-26,F,MUMBAI,973.46,2016-02-08,1970-01-01 00:00:00.000173806,566.0,32.0
4,T8,C1220223,1982-01-27,M,MUMBAI,95075.54,2016-02-08,1970-01-01 00:00:00.000170537,148.0,42.0


In [36]:
# View details about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950938 entries, 0 to 950937
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   TransactionID          950938 non-null  object 
 1   CustomerID             950938 non-null  object 
 2   CustomerDOB            950938 non-null  object 
 3   CustGender             950052 non-null  object 
 4   CustLocation           950827 non-null  object 
 5   CustAccountBalance     950938 non-null  float64
 6   TransactionDate        950938 non-null  object 
 7   TransactionTime        950938 non-null  object 
 8   TransactionAmount (R)  950938 non-null  float64
 9   Age                    950938 non-null  float64
dtypes: float64(3), object(7)
memory usage: 72.6+ MB


In [37]:
# Rename columns
df['Gender'] = df['CustGender']
df['Location'] = df['CustLocation']
df['AccountBalance'] = df['CustAccountBalance']
df.drop(columns=['CustGender', 'CustAccountBalance', 'CustLocation'], inplace=True)

# Fix data types
df['AccountBalance'] = df['AccountBalance'].astype(int)
df['TransactionTime'] = pd.to_datetime(df['TransactionTime'])
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format="%Y-%m-%d")
df['CustomerDOB'] =  pd.to_datetime(df['CustomerDOB'], format="%Y-%m-%d")

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950938 entries, 0 to 950937
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   TransactionID          950938 non-null  object        
 1   CustomerID             950938 non-null  object        
 2   CustomerDOB            950938 non-null  datetime64[ns]
 3   TransactionDate        950938 non-null  datetime64[ns]
 4   TransactionTime        950938 non-null  datetime64[ns]
 5   TransactionAmount (R)  950938 non-null  float64       
 6   Age                    950938 non-null  float64       
 7   Gender                 950052 non-null  object        
 8   Location               950827 non-null  object        
 9   AccountBalance         950938 non-null  int32         
dtypes: datetime64[ns](3), float64(2), int32(1), object(4)
memory usage: 68.9+ MB


In [39]:
# Check N/A
df.isna().sum()

TransactionID              0
CustomerID                 0
CustomerDOB                0
TransactionDate            0
TransactionTime            0
TransactionAmount (R)      0
Age                        0
Gender                   886
Location                 111
AccountBalance             0
dtype: int64

In [49]:
# Check duplicates
print(f"Get the total number of duplicate rows is = {df.duplicated().sum()}")

Get the total number of duplicate rows is = 0


In [54]:
# From 2nd previous cell, notice only two columns have missing values [Gender and Locations]
# view rows with missing values for gender
df[df['Gender'].isna()].head(2)

,TransactionID,CustomerID,CustomerDOB,TransactionDate,TransactionTime,TransactionAmount (R),Age,Gender,Location,AccountBalance
163,T177,C5698953,1800-01-01,2016-12-08,1970-01-01 00:00:00.000155727,3449.00,224.0,NaN,NAVI MUMBAI,8512
1968,T2125,C4525944,1800-01-01,2016-10-21,1970-01-01 00:00:00.000201052,584.06,224.0,NaN,BANGALORE,7115


In [53]:
# view rows with missing values for Location
df[df['Location'].isna()].head(2)

,TransactionID,CustomerID,CustomerDOB,TransactionDate,TransactionTime,TransactionAmount (R),Age,Gender,Location,AccountBalance
2092,T2257,C7383573,1990-01-11,2016-10-21,1970-01-01 00:00:00.000204113,96.0,34.0,M,NaN,262645
32199,T35569,C2737330,1986-10-12,2016-09-26,1970-01-01 00:00:00.000190506,99.0,38.0,M,NaN,17000


In [56]:
import numpy as np
# Divide by CustomerID column since there are no missing values for CustomerID column. Use this column for both Gender and Location

count_in_percentage_gender = np.round(df.isna().sum()["Gender"]/df.CustomerID.shape[0] * 100, 2)
print(f"percentage value of gender rows with missing data is = {count_in_percentage} ")
# Its not a lot of rows that have missing values, so I can drop these rows.

percentage value of gender rows with missing data is = 0.09 


In [57]:
count_in_percentage_location = np.round(df.isna().sum()["Location"]/df.CustomerID.shape[0] * 100, 2)
print(f"percentage value of location rows with missing data is = {count_in_percentage} ")
# Its not a lot of rows that have missing values, so I can drop these rows.

percentage value of location rows with missing data is = 0.09 


In [21]:
# Drop rows where Gender = NaN and where Location is NaN
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 949941 entries, 0 to 950937
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   TransactionID          949941 non-null  object        
 1   CustomerID             949941 non-null  object        
 2   CustomerDOB            949941 non-null  datetime64[ns]
 3   TransactionDate        949941 non-null  datetime64[ns]
 4   TransactionTime        949941 non-null  datetime64[ns]
 5   TransactionAmount (R)  949941 non-null  float64       
 6   Age                    949941 non-null  float64       
 7   Gender                 949941 non-null  object        
 8   Location               949941 non-null  object        
 9   AccountBalance         949941 non-null  int32         
 10  TransactionFrequency   949941 non-null  int64         
dtypes: datetime64[ns](3), float64(2), int32(1), int64(1), object(4)
memory usage: 83.3+ MB


## Feature Engineering

add new features that give further insight about this data. These new features will enhance constructing a reliable learning model.

In [29]:
#Create a new column to track how often a custom transacts.
df['TransactionFrequency'] = df.groupby('CustomerID')['TransactionDate'].transform('nunique')

In [34]:
# Determine when do most transaction happen.
pd.DataFrame(df.TransactionDate.describe())

,TransactionDate
count,949941
mean,2016-07-25 13:31:19.055225600
min,2016-01-08 00:00:00
25%,2016-06-09 00:00:00
50%,2016-08-20 00:00:00
75%,2016-09-09 00:00:00
max,2016-12-09 00:00:00


most tranaction happened in  (MAX = 2016-12-09)